<a href="https://colab.research.google.com/github/brenoslivio/CompVis_ICMC/blob/main/Pratica_03/Pratica_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula Prática 03 - Visualização interativa

**SCC0252/5836 - Visualização Computacional (2022/2)**

Monitor:    Breno Lívio Silva de Almeida, brenoslivio@usp.br

Docente:    Maria Cristina Ferreira de Oliveira, cristina@icmc.usp.br

Material base de Eric Macedo Cabral.

[Repositório no GitHub](https://github.com/brenoslivio/CompVis_ICMC).

---

## Motivação

- Permitir ao usuário interagir com os dados sendo visualizados, pode ampliar as oportunidades de
obter novos insights

- O usuário pode explorar várias perspectivas sobre os dados
    - Alternar entre diferentes mapeamentos

- O usuário pode decidir o que é mais importante na visualização
    - Filtrar ou ressaltar dados
    
- Explorar os dispositivos de interação (teclado, mouse, tela, etc)

Trabalharemos com: 

- Incorporação de elementos de interface ao Jupyter Notebook
- Como fazer a comunicação entre elementos de interface
- Controlar parâmetros de visualização por meio da interação
- Técnicas de interação

## Ferramentas

Popularidade de ferramentas para geração de Dashboards:

![](https://global-uploads.webflow.com/5d3ec351b1eba4332d213004/5f99e10dafbd69a99c875340_C8_qX8dvzv60T4LVZ9GftX-ZH-VJzq3sjUroWWH5XSWw8RFHnCCPPrC6jB3EFVuQdwiqhoEMQKFV-dFz7t6fqaRpSZGvBKI0i1Utj38_j9a54GXMuzi1BiepdIMjOK4ATVdF2131.png)

Fonte da imagem: [Data Revenue](https://www.datarevenue.com/en-blog/data-dashboarding-streamlit-vs-dash-vs-shiny-vs-voila)

Iremos trabalhar principalmente com o ipywidgets com o Voilà, tendo a intenção de transformar um Jupyter Notebook em um Dashboard interativo.

### ipywidgets

[Documentação do ipywidgets](https://ipywidgets.readthedocs.io/)

- Elementos HTML incorporados ao Jupyter Notebook
- Interativos e programáveis
- Baseado em eventos
    - Interação do usuário
    - Mudança do valor de alguma variável
    - Término do processamento de um algoritmo

## Elementos de interface

Existem diversos elementos de interface para o ipywidgets. [Lista de elementos de interface](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html).

Vamos carregar o dataset Gapminder para ver o funcionamento desses elementos.

In [1]:
import pandas as pd
import io
from ipywidgets import widgets
from google.colab import output
output.enable_custom_widget_manager()

df = pd.read_csv('https://raw.githubusercontent.com/brenoslivio/CompVis_ICMC/main/Pratica_03/data/GapMinder.csv')

df["year"] = df["year"].astype("category")
df["country"] = df["country"].astype("category")
df["continent"] = df["continent"].astype("category")

df.head(5)

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4


### Caixa de texto e Botões

- Entrada do usuário
- String
- Query

    - Ações
    
    - Eventos
    
```python
from ipywidgets import widgets

widgets.Text(
    value="Default",
    placeholder="Dica",
    description="Tooltip")

widgets.Button(
    description="Label",
    button_style='',
    icon="search",
    tooltip="")
```

Vamos fazer um sistema de busca para verificar o código alpha-3 para um dado país:

In [2]:
wid_query = {
    "query": widgets.Text(
        value="",
        placeholder="Type a country's name",
        description="Query: ",
        disabled=False),
    "button": widgets.Button(
        description='Search',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        icon='search', # (FontAwesome names without the `fa-` prefix)
        tooltip='Search for a country'),
    "output": widgets.Label(value="")}

def on_query(button):
    wid_query["output"].value = df[df['country'] == wid_query["query"].value]['iso_alpha'].iat[0]
    
# Eventos
wid_query["button"].on_click(on_query)

widgets.VBox([
    widgets.HBox([wid_query["query"], wid_query["button"]]),
    wid_query["output"]])

### Dropdown

Permite que o usuário selecione um valor dentre um conjunto de opções pré-definidas.

```python
from ipywidgets import widgets

widgets.Dropdown(
    options=[Opções],
    value="Default",
    description="Label")
```

Baseado no continente, vamos ver o número de países:

In [3]:
continents = df["continent"].unique().tolist()
text = 'Número de países: '

wid_continents = {
    "dropdown": widgets.Dropdown(
        options= ["All"] + continents,
        value="All",
        description="Continent",
        disabled=False),
    "output": widgets.Label(value=text + str(df['country'].nunique()))}

def select_continent(dropdown):
    # Novo valor acessível por dropdown["new"]
    # Antigo valor acessível por dropdown["old"]
    
    wid_continents["output"].value = text + str(df[df['continent'] == dropdown["new"]]['country'].nunique()) \
                                        if dropdown["new"] != "All" else text + str(df['country'].nunique()) 
    
# Eventos
wid_continents["dropdown"].observe(select_continent, names='value')

widgets.VBox([wid_continents["dropdown"], wid_continents["output"]])

### Slider

Valor flutuante dentro de um intervalo pré-definido.

```python
from ipywidgets import widgets

widgets.IntSlider(
    value=5,
    min=0,
    max=10,
    step=1)
```

Vamos fazer um Slider considerando os anos do dataset Gapminder:

In [4]:
years = df["year"].tolist()

wid_years = {
    "play": widgets.Play(
        value=years[0],
        min=years[0],
        max=years[-1],
        step=1,
        interval=500,
        description="Years",
        disabled=False),
    "slider": widgets.IntSlider(
        value=years[0],
        min=years[0],
        max=years[-1],
        step=1)}

widgets.jslink((wid_years["play"], 'value'),
               (wid_years["slider"], 'value'))

widgets.HBox([wid_years["play"], wid_years["slider"]])

## Layout

[Documentação: Layout](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#Container/Layout-widgets)

[Documentação: Templates](https://ipywidgets.readthedocs.io/en/latest/examples/Layout%20Templates.html)

### Caixas

- Distribuição de elementos sequencialmente
- Horizontal ou verticalmente
- Aninhamento
    - Hierarquias
    
```python
from ipywidgets import widgets

# Vertical
widgets.VBox([widgets])
# Horizontal
widgets.HBox([widgets])
```
    
Um exemplo de caixa **horizontal**:

In [5]:
widgets.HBox([widgets.Button(description=f'{i}') for i in range(2)])

E **vertical**:

In [6]:
widgets.VBox([widgets.Button(description=f'{i}') for i in range(2)])

### Grade

- Disposição dos elementos numa matriz
- O tamanho de cada elemento é especificado por quantidade de células
    - Linhas + Colunas
     
```python
from ipywidgets import GridspecLayout, widgets

grid = GridspecLayout(rows, cols)
# Adiciona Button na primeira célula
grid[0, 0] = widgets.Button()
```

Vamos criar uma função para retornar um botão dada uma descrição e estilo do botão:

In [7]:
from ipywidgets import GridspecLayout, Layout

def create_expanded_button(description, button_style):
    return widgets.Button(
        description=description,
        button_style=button_style,
        layout=Layout(height='auto', width='auto'))

Um exemplo com **células igualmente distribuidas**:

In [8]:
grid = GridspecLayout(4, 3)

for i in range(4):
    for j in range(3):
        grid[i, j] = create_expanded_button('Button {} - {}'.format(i, j), 'warning')
grid

GridspecLayout(children=(Button(button_style='warning', description='Button 0 - 0', layout=Layout(grid_area='w…

E com **células com proporções diferentes**:

In [9]:
grid = GridspecLayout(4, 3, height='300px')
grid[:3, 1:] = create_expanded_button('One', 'success')
grid[:, 0] = create_expanded_button('Two', 'info')
grid[3, 1] = create_expanded_button('Three', 'warning')
grid[3, 2] = create_expanded_button('Four', 'danger')
grid

GridspecLayout(children=(Button(button_style='success', description='One', layout=Layout(grid_area='widget001'…

### Accordion

- Contêiner com sistema de seções
- Mostra uma seção por vez
     - Collapse
     
```python
from ipywidgets import widgets

widgets.Accordion([widgets])
```

In [10]:
accordion = widgets.Accordion([
    widgets.IntSlider(description="Slider"),
    widgets.Text(description="Text")])

for i, title in enumerate(['Slider', 'Text']):
    accordion.set_title(i, title)

accordion.selected_index = 0
accordion

Accordion(children=(IntSlider(value=0, description='Slider'), Text(value='', description='Text')), _titles={'0…

### Abas

- Estrutura de conteiners não ordenados
- Mostra uma aba por vez

```python
from ipywidgets import widgets

widgets.Tab([widgets])
```

In [11]:
tab_contents = ['P0', 'P1', 'P2', 'P3', 'P4']

tab = widgets.Tab([widgets.Text(description=name) for name in tab_contents])

for i in range(len(tab_contents)):
    tab.set_title(i, f"Aba {i}")

tab.selected_index = 1
tab

### Template: AppLayout

- Template com elementos pré-definidos
    - Geralmente utilizados em aplicações modernas
   
- Responsivo

```python
from ipywidgets import AppLayout

AppLayout(
    header=header_widget,
    left_sidebar=left_widget,
    center=center_widget,
    right_sidebar=right_widget,
    footer=footer_widget)
```

In [12]:
from ipywidgets import AppLayout

header_button = create_expanded_button('Header', 'success')
left_button = create_expanded_button('Left', 'info')
center_button = create_expanded_button('Center', 'warning')
right_button = create_expanded_button('Right', 'info')
footer_button = create_expanded_button('Footer', 'success')

AppLayout(height="300px",
    header=header_button,
    left_sidebar=left_button,
    center=center_button,
    right_sidebar=right_button,
    footer=footer_button)

AppLayout(children=(Button(button_style='success', description='Header', layout=Layout(grid_area='header', hei…

## Técnicas de interação

Utilizaremos o conjunto de dados Gapminder para demonstrar técnicas de interação por meio de um layout que cada elemento vai empregar uma das técnicas.

In [13]:
import plotly.graph_objects as go
import plotly.io as pio
import numpy as np
from ipywidgets import VBox, HBox, widgets
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd

class App(object):
    
    def __init__(self, data: pd.DataFrame):
        
        self.data = data
        
        self.years = sorted(data["year"].unique())
        self.countries = data["country"].cat.categories.tolist()
        self.continents = data["continent"].unique().tolist()
        
        self.cols = ["country", "iso_alpha"]
        
        palette = px.colors.qualitative.Plotly
        self.data["color"] = [
            palette[self.continents.index(continent)]
            for continent in self.data["continent"]]
        
        self.current_df = self.query(self.data, self.years[0])
        
        self.selection = self.current_df.copy(deep=True)
                        
    def show(self):
        self.layout = VBox([
            self.toolbar,                     # Row 1: Toolbar
            self.scatter,                     # Row 2: Scatter plot
            HBox([self.table, self.lines])])  # Row 3: Table + Lines

        return self.layout
    
    def query(self, data, year=None, continent=None):
        data = data.copy(deep=True)

        if year:
            data = data.query("year == @year")
        if continent:
            data = data.query("continent == @continent")

        return data

app = App(df)

### Tabela de itens selecionados

- Utilizaremos uma tabela dinâmica renderizada pelo plotly
- Os elementos da tabela serão atualizados pela interação

[Documentação de tabelas no Plotly](https://plotly.com/python/table/)

In [14]:
app.table = go.FigureWidget(data=[go.Table(
    header=dict(
        values=["País", "ISO"],
        line_color="white",
        fill_color="darkslategray",
        align="left"),
    cells=dict(
        values=[
            app.current_df["country"],
            app.current_df["iso_alpha"]],
       line_color="darkslategray",
       fill_color=[app.data["color"][app.current_df.index]] * 2,
       align=["left", "center"]))],
    layout={
        "width": 600,
        "margin": {"t": 25, "b": 0, "l": 0, "r": 0}})

app.table

FigureWidget({
    'data': [{'cells': {'align': [left, center],
                        'fill': {'color': [['#…

### Gráfico de dispersão

- Nossa visão principal é um gráfico de dispersão dos nossos dados, onde:
    - X: PIB per capita
    - Y: Expectativa de vida
    - Tamanho do ponto: População
    - Cor: Continente

Dessa forma, codificamos 4 variaveis do conjunto de dados em uma única visualização. Porém, a visualização resultante acaba ficando com muitas informações implícitas, exigindo muita carga cognitiva do usuário para analisar todos os 4 mapeamentos visuais de todos os 142 pontos.

#### Interação: Hovering

Adicionamos a interação por hovering para fornecer as informações de cada ponto de dados sob demanda, diminuindo a carga cognitiva do usuário de ter que processar visualmente cada um dos mapeamentos visuais para cada ponto de dados.

[Documentação de Hovering do Plotly](https://plotly.com/python/hover-text-and-formatting/)

In [15]:
app.scatter = go.FigureWidget(px.scatter(
    x=app.current_df["gdpPercap"], y=app.current_df["lifeExp"],
    size=app.current_df["pop"], size_max=55,
    log_x=True, range_x=[100,100000], range_y=[25, 90], # diminui a quantidade de espaço vazio
    hover_name=app.current_df["country"], 
    color_discrete_sequence=px.colors.qualitative.Plotly,
    color=app.current_df["continent"],
    labels={
        "x": "PIB per capita",
        "y": "Expectativa de vida",
        "color": "Continente",
        "size": "População"}))

# Formato do popup de hovering
## Aceita variaveis dos pontos de dados (p.e. x, y, marker.size)
app.scatter.update_traces(
    hovertemplate =
    '''<b> %{hovertext} </b><br>
    População: %{marker.size}<br>
    Expectativa de vida: %{y}<br>
    PIB per capita: %{x}''')

app.scatter.update_layout({
    "margin": {
        "t": 60, "b": 5, "l": 5, "r": 5},
    "legend": {
        "title": "",
        "orientation": "h", "yanchor": "bottom",
        "y": 1.02, "xanchor": "right", "x":1}})

FigureWidget({
    'data': [{'hovertemplate': ('<b> %{hovertext} </b><br>\n    ' ... '}<br>\n    PIB per capit…

### Gráfico de linhas

Para adicionar visualizações facetas, utilizamos subplots com eixos compartilhados.

Neste componente visual, vamos mapear os valores de 3 colunas do nosso conjunto de dados, agregados à coluna *year*, representando assim uma visualização da evolução destes valores ente 1952 e 2007 em intervalos de 5 anos:

* PIB per capita
* Expectativa de vida
* População

#### Interação: Facets

Cada gráfico de linha pode ser visto como uma faceta do nosso conjunto de dados, pois estes são referentes a um mesmo período de tempo, representado pelo eixo X compartilhado, mas representando perspectivas (atributos) diferentes.

[Documentação para subplots no Plotly](https://plotly.com/python/subplots/)

In [16]:
from plotly.subplots import make_subplots

rows = ["lifeExp", "gdpPercap", "pop"] # Facetas exploradas

app.lines = go.FigureWidget(make_subplots(
    shared_xaxes=True,
    specs=[[{"type": "scatter"}],
           [{"type": "scatter"}],
           [{"type": "scatter"}]],
    subplot_titles=("Expectativa de vida", "PIB per capita", "População"),
    rows=len(rows), cols=1))

for country, country_df in app.data.groupby("country"): # Itera pais por pais
    for index, row in enumerate(rows): # Itera por facetas
        app.lines.add_trace(
            go.Scattergl(mode="lines", # Pela quantidade de entidades (426), usamos Scattergl por ter uma melhor performance
                x=country_df["year"],
                y=country_df[row],
                hovertemplate="<b>%{x}</b>: %{y}",
                visible=True,
                marker={
                    "color": app.data["color"][country_df.index[0]],
                    "opacity": 0.75
                }, name=country),
            row=index+1, col=1)

for i in range(len(rows)):
    # Coloca todos os eixos na mesma magnitude
    app.lines.update_xaxes(dict(
        tickmode = 'array',
        tickvals = app.years,
        ticktext = app.years), row=i+1, col=1)
    
app.lines.update_layout({
    "width": 700,
    "margin": {"t": 50, "b": 0, "l": 0, "r": 0},
    "showlegend": False})

FigureWidget({
    'data': [{'hovertemplate': '<b>%{x}</b>: %{y}',
              'marker': {'color': '#636EFA'…

### Elementos de interface

Apenas utilizaremos um *Slider* de anos e um *Player* para percorrer o *Slides* automáticamente, realizando assim uma **Dynamic Query**.

In [17]:
app.player = widgets.Play(
    value=app.years[0],
    min=app.years[0],
    max=app.years[-1],
    step=5,
    interval=1000,
    description=" Ano ",
    disabled=False)

app.slider = widgets.IntSlider(
    value=app.years[0],
    min=app.years[0],
    max=app.years[-1],
    step=5,
    description=' Ano ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

app.toolbar = HBox([app.player, app.slider])

# Faz o link entre o slider e o player
widgets.jslink((app.player, 'value'),
               (app.slider, 'value'))

app.toolbar

### Linking and Brushing

Permite que o usuário utilize ferramentas de seleção (*Brushes*) para ressaltar os elementos de seu interesse. Por meio da abordagem de **Linking and Brushing**, o usuário pode fazer seleções de dados em representações visuais mais simples (p.e. um Scatter plot 2D) e visualizar os resultados em representações mais complexas (p.e. uma visualização facetada).

No nosso caso, fazemos o link das seleções no Scatter plot à nossa tabela e nossa visualização facetada de linhas.
* Scatter plot - Tabela de dados
* Scatter plot - Facetas

In [18]:
def on_selection(trace, points, selector=None):
    # O método on_deselect tem assinatura (trace, points)
    # O método on_select tem assinatura (trace, points, selector)
    from plotly.callbacks import BoxSelector
    
    if selector: # on_select
        if type(selector) == BoxSelector: # Evita a seleção por "Lasso Selection"
            x = selector.xrange # Intervalo selecionado no eixo X
            y = selector.yrange # Intervalo selecionado no eixo Y
        
            app.selection = app.current_df[
                (app.current_df['gdpPercap'].between(x[0], x[1], inclusive=True)) &
                (app.current_df['lifeExp'].between(y[0], y[1], inclusive=True))]
    else: # on_deselect
        app.selection = app.current_df
        
    ## batch_update():
    # Finaliza as operações dentro do block "with" antes de propagar para os gráficos
    with app.table.batch_update():
        # Atualiza os valores da tabela
        app.table.data[0].cells.values = app.selection[app.cols].T
        
        # Atualiza as cores das células:
        app.table.data[0].cells.fill.color = [app.data["color"][app.selection.index]] * 2 
        
        # Atualiza o parâmetro "visible" de todos os traces
        app.lines.plotly_restyle({'visible': [
            trace.name in app.selection["country"].tolist()
            for trace in app.lines.data]})
    
app.on_selection = on_selection

app.scatter.data[0].on_selection(app.on_selection) # Adiciona a callback para seleção
app.scatter.data[0].on_deselect(app.on_selection) # Adiciona a callback para desseleção

VBox([
    app.scatter,
    HBox([app.table, app.lines])])

    'data': [{'hovertemplate': ('<b> %{hovertext} </b><br>\n    ' ... '}<br>\n  …

### Dynamic query

* Ligamos o elemento de interface da barra de ferramentas (*Toolbar*) ao nosso gráfico de dispersão

O usuário pode interagir com o Slider navegando pelos anos (1952-2007) em passos de 5 anos, sem perder informação entre as iterações. O recurso de animação dos dados permite que as mudanças não sejam drásticas e que possam ser acompanhadas pelo usuário.

In [19]:
def on_year_change(year):        
    app.current_df = app.query(app.data, year=year["new"]) # Query por ano

    ## batch_animate():
    # Anima as alterações realizadas dentro do block "with"
    with app.scatter.batch_animate():
        for continent_group in app.scatter.data:
            continent = continent_group.legendgroup
            df_cont = app.query(app.current_df, continent=continent)

            continent_group.marker.size = df_cont["pop"]
            continent_group.x = df_cont["gdpPercap"]
            continent_group.y = df_cont["lifeExp"]

app.on_year_change = on_year_change

app.slider.observe(app.on_year_change, "value")

VBox([app.toolbar, app.scatter])

### Resultado

In [20]:
app.show()